####  KPI Implementation & Analysis

In [9]:

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load cleaned data
df = pd.read_csv('../data/final/movies_with_kpis.csv')

print(f"Dataset loaded: {df.shape}")
print(f"Date range: {df['release_date'].min()} to {df['release_date'].max()}")
print(f"\nFirst few rows:")
df.head()

Dataset loaded: (18, 36)
Date range: 1997-11-18 to 2019-11-20

First few rows:


,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,...,roi,profit_musd,genre_count,is_profitable,budget_musd_is_outlier,revenue_musd_is_outlier,runtime_is_outlier,popularity_is_outlier,completeness_score,is_franchise
0,299534,Avengers: Endgame,Avenge the fallen.,2019-04-24,Adventure|Science Fiction|Action,The Avengers Collection,en,356.0,2799.439100,Marvel Studios,...,686.359298,2443.439100,3,1,0,0,1,0,1.000000,1
1,19995,Avatar,Enter the world of Pandora.,2009-12-15,Action|Adventure|Fantasy|Science Fiction,Avatar Collection,en,237.0,2923.706026,Dune Entertainment|Lightstorm Entertainment|20...,...,1133.631235,2686.706026,4,1,0,1,0,1,1.000000,1
2,140607,Star Wars: The Force Awakens,Every generation has a story.,2015-12-15,Adventure|Action|Science Fiction,Star Wars Collection,en,245.0,2068.223624,Lucasfilm Ltd.|Bad Robot,...,744.172908,1823.223624,3,1,0,0,0,0,1.000000,1
3,299536,Avengers: Infinity War,Destiny arrives all the same.,2018-04-25,Adventure|Action|Science Fiction,The Avengers Collection,en,300.0,2052.415039,Marvel Studios,...,584.138346,1752.415039,3,1,0,0,0,0,1.000000,1
4,597,Titanic,Nothing on earth could come between them.,1997-11-18,Drama|Romance,NaN,en,200.0,2264.162353,Paramount Pictures|20th Century Fox|Lightstorm...,...,1032.081177,2064.162353,2,1,0,0,1,0,0.970588,0


### This notebook covers:
#### 1. Best/Worst performing movies by various metrics
#### 2. Advanced filtering & search queries
#### 3. Franchise vs. Standalone comparison
#### 4. Most successful franchises & directors

In [ ]:
# Ranking Function
def rank_movies(df, metric, top_n=10, ascending=False, min_budget=None, min_votes=None):
    """
    Universal ranking function for movies
    
    Parameters:
    -----------
    df : DataFrame
        Movies dataset
    metric : str
        Column name to rank by
    top_n : int
        Number of top results to return
    ascending : bool
        Sort order (False = highest first)
    min_budget : float
        Minimum budget threshold (in millions)
    min_votes : int
        Minimum vote count threshold
    
    Returns:
    --------
    DataFrame with top N ranked movies
    """
    # creating a copy to avoid modifying original dataframe
    temp_df = df.copy()
    
    # Applying filters
    if min_budget is not None:
        temp_df = temp_df[temp_df['budget_musd'] >= min_budget]
    
    if min_votes is not None:
        temp_df = temp_df[temp_df['vote_count'] >= min_votes]
    
    # Removing rows with NaN in the metric column
    temp_df = temp_df[temp_df[metric].notna()]
    
    # Sort and return top movies
    result = temp_df.sort_values(by=metric, ascending=ascending).head(top_n)
    
    # Defining base columns (to avoid duplicating the metric column)
    base_columns = ['title', 'release_date', 'budget_musd', 'revenue_musd', 
                    'vote_average', 'vote_count', 'genres', 'director']
    
    # Add metric column if it's not already in base_columns
    if metric not in base_columns:
        columns = ['title', 'release_date', metric] + [col for col in base_columns[2:]]
    else:
        # Reorder to put metric after release_date
        columns = ['title', 'release_date', metric] + [col for col in base_columns[2:] if col != metric]
    
    return result[columns]



### 1. Best/Worst Performing Movies
#### Identifying top performers across different metrics

In [18]:
# Highest Revenue Movies
print("=" * 80)
print("TOP 10 HIGHEST REVENUE MOVIES")
print("=" * 80)
highest_revenue = rank_movies(df, 'revenue_musd', top_n=10)
print(highest_revenue.to_markdown(index=False))

TOP 10 HIGHEST REVENUE MOVIES
| title                        | release_date   |   revenue_musd |   budget_musd |   vote_average |   vote_count | genres                                    | director        |
|:-----------------------------|:---------------|---------------:|--------------:|---------------:|-------------:|:------------------------------------------|:----------------|
| Avatar                       | 2009-12-15     |        2923.71 |           237 |          7.594 |        32883 | Action|Adventure|Fantasy|Science Fiction  | James Cameron   |
| Avengers: Endgame            | 2019-04-24     |        2799.44 |           356 |          8.237 |        26978 | Adventure|Science Fiction|Action          | Anthony Russo   |
| Titanic                      | 1997-11-18     |        2264.16 |           200 |          7.903 |        26519 | Drama|Romance                             | James Cameron   |
| Star Wars: The Force Awakens | 2015-12-15     |        2068.22 |           245 |   

In [19]:
# Highest Budget Movies
print("\n" + "=" * 80)
print("TOP 10 HIGHEST BUDGET MOVIES")
print("=" * 80)
highest_budget = rank_movies(df, 'budget_musd', top_n=10)
print(highest_budget.to_markdown(index=False))


TOP 10 HIGHEST BUDGET MOVIES
| title                        | release_date   |   budget_musd |   revenue_musd |   vote_average |   vote_count | genres                                   | director      |
|:-----------------------------|:---------------|--------------:|---------------:|---------------:|-------------:|:-----------------------------------------|:--------------|
| Avengers: Endgame            | 2019-04-24     |           356 |        2799.44 |          8.237 |        26978 | Adventure|Science Fiction|Action         | Anthony Russo |
| Avengers: Infinity War       | 2018-04-25     |           300 |        2052.42 |          8.235 |        31188 | Adventure|Action|Science Fiction         | Joe Russo     |
| Star Wars: The Last Jedi     | 2017-12-13     |           300 |        1332.7  |          6.8   |        15928 | Adventure|Action|Science Fiction         | Rian Johnson  |
| The Lion King                | 2019-07-12     |           260 |        1662.02 |          7.102 | 

In [ ]:
# Highest Profit Movies  (Profit = Revenue - Budget)
print("\n" + "=" * 80)
print("TOP 10 HIGHEST PROFIT MOVIES")
print("=" * 80)
highest_profit = rank_movies(df, 'profit_musd', top_n=10)
print(highest_profit.to_markdown(index=False))


TOP 10 HIGHEST PROFIT MOVIES
| title                        | release_date   |   profit_musd |   budget_musd |   revenue_musd |   vote_average |   vote_count | genres                                    | director        |
|:-----------------------------|:---------------|--------------:|--------------:|---------------:|---------------:|-------------:|:------------------------------------------|:----------------|
| Avatar                       | 2009-12-15     |       2686.71 |           237 |        2923.71 |          7.594 |        32883 | Action|Adventure|Fantasy|Science Fiction  | James Cameron   |
| Avengers: Endgame            | 2019-04-24     |       2443.44 |           356 |        2799.44 |          8.237 |        26978 | Adventure|Science Fiction|Action          | Anthony Russo   |
| Titanic                      | 1997-11-18     |       2064.16 |           200 |        2264.16 |          7.903 |        26519 | Drama|Romance                             | James Cameron   |
| Sta

In [ ]:
# Lowest Profit Movies (revenue - budget)
print("\n" + "=" * 80)
print("TOP 10 LOWEST PROFIT MOVIES (BIGGEST LOSSES)")
print("=" * 80)
lowest_profit = rank_movies(df, 'profit_musd', top_n=10, ascending=True)
print(lowest_profit.to_markdown(index=False))


TOP 10 LOWEST PROFIT MOVIES (BIGGEST LOSSES)
| title                                        | release_date   |   profit_musd |   budget_musd |   revenue_musd |   vote_average |   vote_count | genres                                    | director     |
|:---------------------------------------------|:---------------|--------------:|--------------:|---------------:|---------------:|-------------:|:------------------------------------------|:-------------|
| Star Wars: The Last Jedi                     | 2017-12-13     |       1032.7  |           300 |        1332.7  |          6.8   |        15928 | Adventure|Action|Science Fiction          | Rian Johnson |
| Incredibles 2                                | 2018-06-14     |       1043.23 |           200 |        1243.23 |          7.455 |        13373 | Action|Adventure|Animation|Family         | Brad Bird    |
| Beauty and the Beast                         | 2017-03-16     |       1106.12 |           160 |        1266.12 |          6.969 

In [ ]:
# Highest ROI Movies (Budget >= $10M)     Revenue / Budget
print("\n" + "=" * 80)
print("TOP 10 HIGHEST ROI MOVIES (Budget >= $10M)")
print("=" * 80)
highest_roi = rank_movies(df, 'roi', top_n=10, min_budget=10)
print(highest_roi.to_markdown(index=False))


TOP 10 HIGHEST ROI MOVIES (Budget >= $10M)
| title                                        | release_date   |      roi |   budget_musd |   revenue_musd |   vote_average |   vote_count | genres                                    | director        |
|:---------------------------------------------|:---------------|---------:|--------------:|---------------:|---------------:|-------------:|:------------------------------------------|:----------------|
| Avatar                                       | 2009-12-15     | 1133.63  |           237 |        2923.71 |          7.594 |        32883 | Action|Adventure|Fantasy|Science Fiction  | James Cameron   |
| Titanic                                      | 1997-11-18     | 1032.08  |           200 |        2264.16 |          7.903 |        26519 | Drama|Romance                             | James Cameron   |
| Jurassic World                               | 2015-06-06     | 1014.36  |           150 |        1671.54 |          6.699 |        21127 

In [23]:
#Lowest ROI Movies (Budget >= $10M)
print("\n" + "=" * 80)
print("TOP 10 LOWEST ROI MOVIES (Budget >= $10M)")
print("=" * 80)
lowest_roi = rank_movies(df, 'roi', top_n=10, min_budget=10, ascending=True)
print(lowest_roi.to_markdown(index=False))


TOP 10 LOWEST ROI MOVIES (Budget >= $10M)
| title                          | release_date   |     roi |   budget_musd |   revenue_musd |   vote_average |   vote_count | genres                                    | director      |
|:-------------------------------|:---------------|--------:|--------------:|---------------:|---------------:|-------------:|:------------------------------------------|:--------------|
| Star Wars: The Last Jedi       | 2017-12-13     | 344.233 |           300 |        1332.7  |          6.8   |        15928 | Adventure|Action|Science Fiction          | Rian Johnson  |
| Avengers: Age of Ultron        | 2015-04-22     | 498.044 |           235 |        1405.4  |          7.271 |        23882 | Action|Adventure|Science Fiction          | Joss Whedon   |
| Incredibles 2                  | 2018-06-14     | 521.613 |           200 |        1243.23 |          7.455 |        13373 | Action|Adventure|Animation|Family         | Brad Bird     |
| The Lion King       

In [24]:
#  Most Voted Movies
print("\n" + "=" * 80)
print("TOP 10 MOST VOTED MOVIES")
print("=" * 80)
most_voted = rank_movies(df, 'vote_count', top_n=10)
print(most_voted.to_markdown(index=False))


TOP 10 MOST VOTED MOVIES
| title                                        | release_date   |   vote_count |   budget_musd |   revenue_musd |   vote_average | genres                                    | director        |
|:---------------------------------------------|:---------------|-------------:|--------------:|---------------:|---------------:|:------------------------------------------|:----------------|
| The Avengers                                 | 2012-04-25     |        34325 |           220 |        1518.82 |          7.87  | Science Fiction|Action|Adventure          | Joss Whedon     |
| Avatar                                       | 2009-12-15     |        32883 |           237 |        2923.71 |          7.594 | Action|Adventure|Fantasy|Science Fiction  | James Cameron   |
| Avengers: Infinity War                       | 2018-04-25     |        31188 |           300 |        2052.42 |          8.235 | Adventure|Action|Science Fiction          | Joe Russo       |
| Avenger

In [25]:
# Highest Rated Movies (>= 10 votes)
print("\n" + "=" * 80)
print("TOP 10 HIGHEST RATED MOVIES (>= 10 votes)")
print("=" * 80)
highest_rated = rank_movies(df, 'vote_average', top_n=10, min_votes=10)
print(highest_rated.to_markdown(index=False))


TOP 10 HIGHEST RATED MOVIES (>= 10 votes)
| title                                        | release_date   |   vote_average |   budget_musd |   revenue_musd |   vote_count | genres                                   | director      |
|:---------------------------------------------|:---------------|---------------:|--------------:|---------------:|-------------:|:-----------------------------------------|:--------------|
| Avengers: Endgame                            | 2019-04-24     |          8.237 |           356 |        2799.44 |        26978 | Adventure|Science Fiction|Action         | Anthony Russo |
| Avengers: Infinity War                       | 2018-04-25     |          8.235 |           300 |        2052.42 |        31188 | Adventure|Action|Science Fiction         | Joe Russo     |
| Harry Potter and the Deathly Hallows: Part 2 | 2011-07-12     |          8.084 |           125 |        1341.51 |        21464 | Adventure|Fantasy                        | David Yates   |
| Titan

In [26]:
# Lowest Rated Movies (>= 10 votes)
print("\n" + "=" * 80)
print("TOP 10 LOWEST RATED MOVIES (>= 10 votes)")
print("=" * 80)
lowest_rated = rank_movies(df, 'vote_average', top_n=10, min_votes=10, ascending=True)
print(lowest_rated.to_markdown(index=False))


TOP 10 LOWEST RATED MOVIES (>= 10 votes)
| title                          | release_date   |   vote_average |   budget_musd |   revenue_musd |   vote_count | genres                                    | director        |
|:-------------------------------|:---------------|---------------:|--------------:|---------------:|-------------:|:------------------------------------------|:----------------|
| Jurassic World: Fallen Kingdom | 2018-06-06     |          6.537 |           170 |        1310.47 |        12413 | Action|Adventure|Science Fiction|Thriller | J.A. Bayona     |
| Jurassic World                 | 2015-06-06     |          6.699 |           150 |        1671.54 |        21127 | Action|Adventure|Science Fiction|Thriller | Colin Trevorrow |
| Star Wars: The Last Jedi       | 2017-12-13     |          6.8   |           300 |        1332.7  |        15928 | Adventure|Action|Science Fiction          | Rian Johnson    |
| Beauty and the Beast           | 2017-03-16     |          6.

In [27]:
# Most Popular Movies
print("\n" + "=" * 80)
print("TOP 10 MOST POPULAR MOVIES")
print("=" * 80)
most_popular = rank_movies(df, 'popularity', top_n=10)
print(most_popular.to_markdown(index=False))


TOP 10 MOST POPULAR MOVIES
| title                                        | release_date   |   popularity |   budget_musd |   revenue_musd |   vote_average |   vote_count | genres                                   | director      |
|:---------------------------------------------|:---------------|-------------:|--------------:|---------------:|---------------:|-------------:|:-----------------------------------------|:--------------|
| The Avengers                                 | 2012-04-25     |      40.3021 |           220 |        1518.82 |          7.87  |        34325 | Science Fiction|Action|Adventure         | Joss Whedon   |
| Avatar                                       | 2009-12-15     |      38.2316 |           237 |        2923.71 |          7.594 |        32883 | Action|Adventure|Fantasy|Science Fiction | James Cameron |
| Titanic                                      | 1997-11-18     |      23.4289 |           200 |        2264.16 |          7.903 |        26519 | Drama|

### 2. Advanced Movie Filtering & Search Queries
#### Complex queries combining multiple criteria

In [28]:
# Search 1 - Best-rated Science Fiction Action movies starring Bruce Willis
print("=" * 80)
print("SEARCH 1: Best Sci-Fi Action Movies with Bruce Willis")
print("=" * 80)

# Filter criteria
search1 = df[
    (df['genres'].str.contains('Science Fiction', case=False, na=False)) &
    (df['genres'].str.contains('Action', case=False, na=False)) &
    (df['cast'].str.contains('Bruce Willis', case=False, na=False))
].sort_values(by='vote_average', ascending=False)

# Display results
if len(search1) > 0:
    print(search1[['title', 'vote_average', 'genres', 'cast', 'release_date']].to_string(index=False))
else:
    print("No movies found matching criteria")

SEARCH 1: Best Sci-Fi Action Movies with Bruce Willis
No movies found matching criteria


In [37]:
# Search 2 - Uma Thurman movies directed by Quentin Tarantino
print("\n" + "=" * 80)
print("SEARCH 2: Uma Thurman Movies Directed by Quentin Tarantino")
print("=" * 80)

# Filter criteria
search2 = df[
    (df['cast'].str.contains('Uma Thurman', case=False, na=False)) &
    (df['director'].str.contains('Quentin Tarantino', case=False, na=False))
].sort_values(by='runtime', ascending=True)

# Display results
if len(search2) > 0:
    print(search2[['title', 'runtime', 'director', 'cast', 'release_date', 'vote_average']].to_markdown(index=False))
else:
    print("No movies found matching criteria")


SEARCH 2: Uma Thurman Movies Directed by Quentin Tarantino
No movies found matching criteria


### 3. Franchise vs. Standalone Movie Performance
#### Comparing collection movies vs standalone releases

In [39]:
# Franchise vs Standalone Comparison
print("=" * 80)
print("FRANCHISE VS. STANDALONE MOVIES COMPARISON")
print("=" * 80)

# Comparison
comparison = df.groupby('is_franchise').agg({
    'revenue_musd': 'mean',
    'roi': 'median',
    'budget_musd': 'mean',
    'popularity': 'mean',
    'vote_average': 'mean',
    'title': 'count'
}).round(2)

comparison.columns = ['Mean Revenue ($M)', 'Median ROI (%)', 'Mean Budget ($M)', 
                      'Mean Popularity', 'Mean Rating', 'Movie Count']
comparison.index = ['Standalone', 'Franchise']

print(comparison)

# Insights
print("\n" + "=" * 80)
print("INSIGHTS:")
print("=" * 80)
revenue_diff = comparison.loc['Franchise', 'Mean Revenue ($M)'] - comparison.loc['Standalone', 'Mean Revenue ($M)']
print(f"Franchise movies earn ${revenue_diff:.2f}M more on average")
print(f"Franchise count: {comparison.loc['Franchise', 'Movie Count']:.0f}")
print(f"Standalone count: {comparison.loc['Standalone', 'Movie Count']:.0f}")

FRANCHISE VS. STANDALONE MOVIES COMPARISON
            Mean Revenue ($M)  Median ROI (%)  Mean Budget ($M)  \
Standalone            1765.14          861.70             180.0   
Franchise             1682.67          678.61             218.0   

            Mean Popularity  Mean Rating  Movie Count  
Standalone            16.63         7.44            2  
Franchise             15.00         7.39           16  

INSIGHTS:
Franchise movies earn $-82.47M more on average
Franchise count: 16
Standalone count: 2


### 4. Most Successful Franchises
#### Analyzing franchise performance metrics

In [42]:
# Top Franchises Analysis
print("=" * 80)
print("TOP 10 MOST SUCCESSFUL FRANCHISES")
print("=" * 80)

# Filter only franchise movies
franchise_movies = df[df['belongs_to_collection'].notna()].copy()

# Group by franchise
franchise_stats = franchise_movies.groupby('belongs_to_collection').agg({
    'title': 'count',
    'budget_musd': ['sum', 'mean'],
    'revenue_musd': ['sum', 'mean'],
    'vote_average': 'mean'
}).round(2)

# Flatten column names
franchise_stats.columns = ['Movie_Count', 'Total_Budget', 'Mean_Budget', 
                            'Total_Revenue', 'Mean_Revenue', 'Mean_Rating']

# Sort by total revenue
franchise_stats = franchise_stats.sort_values(by='Total_Revenue', ascending=False).head(10)

print(franchise_stats.to_markdown())

TOP 10 MOST SUCCESSFUL FRANCHISES
| belongs_to_collection               |   Movie_Count |   Total_Budget |   Mean_Budget |   Total_Revenue |   Mean_Revenue |   Mean_Rating |
|:------------------------------------|--------------:|---------------:|--------------:|----------------:|---------------:|--------------:|
| The Avengers Collection             |             4 |           1111 |        277.75 |         7776.07 |        1944.02 |          7.9  |
| Star Wars Collection                |             2 |            545 |        272.5  |         3400.92 |        1700.46 |          7.05 |
| Jurassic Park Collection            |             2 |            320 |        160    |         2982.01 |        1491    |          6.62 |
| Avatar Collection                   |             1 |            237 |        237    |         2923.71 |        2923.71 |          7.59 |
| Frozen Collection                   |             2 |            300 |        150    |         2727.9  |        1363.95 |   

### 5. Most Successful Directors
#### Ranking directors by their filmography performance

In [43]:
# Top Directors Analysis
print("=" * 80)
print("TOP 10 MOST SUCCESSFUL DIRECTORS")
print("=" * 80)

# Filter movies with directors
director_movies = df[df['director'].notna()].copy()

# Group by director
director_stats = director_movies.groupby('director').agg({
    'title': 'count',
    'revenue_musd': 'sum',
    'vote_average': 'mean'
}).round(2)

director_stats.columns = ['Total_Movies', 'Total_Revenue_$M', 'Mean_Rating']

# Sort by total revenue
director_stats = director_stats.sort_values(by='Total_Revenue_$M', ascending=False).head(10)

print(director_stats.to_markdown())

TOP 10 MOST SUCCESSFUL DIRECTORS
| director        |   Total_Movies |   Total_Revenue_$M |   Mean_Rating |
|:----------------|---------------:|-------------------:|--------------:|
| James Cameron   |              2 |            5187.87 |          7.75 |
| Joss Whedon     |              2 |            2924.22 |          7.57 |
| Anthony Russo   |              1 |            2799.44 |          8.24 |
| J.J. Abrams     |              1 |            2068.22 |          7.3  |
| Joe Russo       |              1 |            2052.42 |          8.24 |
| Colin Trevorrow |              1 |            1671.54 |          6.7  |
| Jon Favreau     |              1 |            1662.02 |          7.1  |
| James Wan       |              1 |            1515.4  |          7.22 |
| Jennifer Lee    |              1 |            1453.68 |          7.24 |
| Ryan Coogler    |              1 |            1349.93 |          7.37 |


In [44]:
# KPI Results
import os

#  KPI summary
kpi_summary = {
    'Highest Revenue': highest_revenue.head(5),
    'Highest ROI': highest_roi.head(5),
    'Highest Rated': highest_rated.head(5),
    'Franchise vs Standalone': comparison
}

# Exporting  to reports
with open('../reports/kpi_detailed_results.txt', 'w') as f:
    f.write("TMDB MOVIE DATA ANALYSIS - DETAILED KPI RESULTS\n")
    f.write("=" * 80 + "\n\n")
    
    for title, data in kpi_summary.items():
        f.write(f"\n{title}:\n")
        f.write("-" * 80 + "\n")
        f.write(data.to_string())
        f.write("\n\n")

print("KPI analysis complete! Results saved to ../reports/kpi_detailed_results.txt")

KPI analysis complete! Results saved to ../reports/kpi_detailed_results.txt
